Prerequisites
=============

As referred to below, `Index.h` is the libclang API header, available within the Clang source tree at `include/clang-c/Index.h` ([suggested download location](https://raw.github.com/llvm-mirror/clang/master/include/clang-c/Index.h)).

The following definitions should be adjusted to match the appropriate paths on your system, if libclang is installed separately from Julia.

In [1]:
indexh         = joinpath(JULIA_HOME, "../include/clang-c/Index.h")
clang_includes = [joinpath(JULIA_HOME, "../lib/clang/3.3/include"), joinpath(dirname(indexh), "..")]

2-element Array{UTF8String,1}:
 "/Users/jacobbolewski/Julia/julia/usr/bin/../lib/clang/3.3/include"
 "/Users/jacobbolewski/Julia/julia/usr/bin/../include/clang-c/.."   

In [3]:
@assert false

LoadError: LoadError: AssertionError: false
while loading In[3], in expression starting on line 1

Printing Enums
==============

In [4]:
using Clang.cindex

top = parse_header(indexh; includes=clang_includes, diagnostics=true)

function print_enums(enumdef::EnumDecl)
    for enum in children(enumdef)
        println("  const ", name(enum), " = ", value(enum))
    end
end

for cursor in children(top)
    # Skip cursors not in target file
    if (basename(cu_file(cursor)) != basename(indexh)) continue end
    
    got_enum = false
    if isa(cursor, EnumDecl)
        #if (name(cursor) == "") continue end
        println("# Enum: ", name(cursor))
        print_enums(cursor)
        got_enum = true
    elseif isa(cursor, TypedefDecl)
        td_children = children(cursor)
        td_children.size == 0 && continue
        
        td_cursor = td_children[1]
        if isa(td_cursor, EnumDecl)
            println("# Typedef Enum: ", name(td_cursor))
            print_enums(td_cursor)
            got_enum = true
        end
    end
end

LoadError: LoadError: MethodError: `tu_parse` has no method matching tu_parse(::Ptr{Void}, ::UTF8String, ::Array{UTF8String,1}, ::Int64, ::Ptr{Void}, ::Int64, ::Int64)
Closest candidates are:
  tu_parse(::Any, !Matched::ASCIIString, !Matched::Array{ASCIIString,1}, ::Any, ::Ptr{Void}, ::Any, ::Any)
while loading In[4], in expression starting on line 3

Print Functions
===============

Expected output:

```
Number of cursors in TranslationUnit: 679
Functions:
  clang_createIndex(CXIndex,Int,Int,)
  clang_disposeIndex(Typedef,)
  clang_CXIndex_setGlobalOptions(Typedef,UInt,)
  clang_CXIndex_getGlobalOptions(Typedef,)
  clang_getFileName(CXString,Typedef,)
  clang_getFileTime(time_t,Typedef,)
  ...
```

Note: number of cursors may vary depending on the definitions activated in previous includes based on system type and configuration.

In [5]:
using Clang.cindex

get_it(t::TypedefDecl) = get_it(cindex.getTypedefDeclUnderlyingType(t))
get_it(d::ParmDecl)    = get_it(cu_type(d))
get_it(t::TypeRef)     = spelling(t)
get_it(t::Pointer) = string("Ptr{", get_it(cindex.getPointeeType(t)), "}")
get_it(t::CLType)  = spelling(t)

get_it(c::CLCursor) = get_it(cindex.getCursorType(c))
get_it(f::FunctionDecl) = spelling(f)

function print_function(f::FunctionDecl)
    rt = return_type(f)
    print("  ", get_it(f))
    print("(")
    for a in children(f)
        if !(isa(a, TypeRef) || isa(a, ParmDecl)) break end
        print(get_it(a))
        print(",")
    end
    print(")\n")
end

function dump_functions(header::String)
    tu = parse_header(header; includes=clang_includes)
    tu_children = children(tu)
    
    println("Number of cursors in TranslationUnit: ", length(tu_children))
    println("Functions:")
    for node in children(tu)
        # Skip anything outside of the TU
        if (basename(cu_file(node)) != basename(header))
            continue
        end
        if isa(node, FunctionDecl)
            print_function(node)
        end
    end
end

dump_functions(indexh)

LoadError: LoadError: MethodError: `tu_parse` has no method matching tu_parse(::Ptr{Void}, ::UTF8String, ::Array{UTF8String,1}, ::Int64, ::Ptr{Void}, ::Int64, ::Int64)
Closest candidates are:
  tu_parse(::Any, !Matched::ASCIIString, !Matched::Array{ASCIIString,1}, ::Any, ::Ptr{Void}, ::Any, ::Any)
while loading In[5], in expression starting on line 41

Printing Structs
================

Expected output:
```
Struct: CXTranslationUnitImpl
Struct: CXUnsavedFile
Struct: CXVersion
Struct: CXVersion
Struct: 
Struct: 
```
Note: it takes a little bit more work to get the name of typedef'd structs. See Clang.jl/src/wrap_c.jl for a full implementation.

In [11]:
using Clang.cindex

get_struct(t::TypedefDecl) = get_struct(cindex.getTypeDeclaration(cindex.getTypedefDeclUnderlyingType(t)))
get_struct(d::StructDecl)  = d
get_struct(d::CLCursor)    = None

function dump_structs(hdr::String)
    topcu = parse_header(hdr; includes = clang_includes)
    for cu in children(topcu)
        # Skip includes
        if (basename(cu_file(cu)) != basename(hdr)) continue end
        cur = get_struct(cu)
        if (cur != None)
            println("Struct: ", name(cur))
        end
    end
end

dump_structs(indexh)

Struct: CXTranslationUnitImpl
Struct: CXUnsavedFile
Struct: CXVersion
Struct: CXVersion
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: CXTUResourceUsageEntry
Struct: CXTUResourceUsageEntry
Struct: CXTUResourceUsage
Struct: CXTUResourceUsage
Struct: 
Struct: 
Struct: 
Struct: 
Struct: CXPlatformAvailability
Struct: CXPlatformAvailability
Struct: CXCursorSetImpl
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
Struct: 
